In [1]:
# Import libraries
from langchain_community.graphs import Neo4jGraph
from graph_rag_query import GraphRAGQuery
from graph_rag_embeddings import EntityEmbeddings
from graph_rag_hybrid import HybridRetriever
import json

/home/duo/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup Neo4j connection
NEO4J_URI = "neo4j+s://0c367113.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gTO1K567hBLzkRdUAhhEb-UqvBjz0i3ckV3M9v_-Nio"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print("Kết nối Neo4j thành công!")

/tmp/ipykernel_651057/502800272.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Kết nối Neo4j thành công!


## Initialize Components

Load các components từ Bước 1 và Bước 2

In [3]:
# Initialize Bước 1: Graph Queries
print("Loading Graph Query component...")
graph_query = GraphRAGQuery(graph)
print("Graph queries ready")

# Initialize Bước 2: Embeddings
print("\nLoading Embeddings component...")
embeddings = EntityEmbeddings(graph)
print("Embeddings ready")

Loading Graph Query component...
Graph queries ready

Loading Embeddings component...
Loading embedding model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Model loaded! Embedding dimension: 768
Embeddings ready


In [4]:
# Initialize Bước 3: Hybrid Retriever
print("Initializing Hybrid Retriever...")
hybrid = HybridRetriever(graph_query, embeddings)
print("Hybrid Retriever ready!")

Initializing Hybrid Retriever...
Hybrid Retriever ready!


## Test Basic Retrieval

In [5]:
# Test với câu hỏi đơn giản
question = "Ai là người lãnh đạo kháng chiến chống Pháp?"

print(f"Question: {question}\n")

context = hybrid.retrieve(
    question=question,
    top_k=10,
    vector_top_k=5,
    expansion_depth=1
)

Question: Ai là người lãnh đạo kháng chiến chống Pháp?

Query: Ai là người lãnh đạo kháng chiến chống Pháp?
Question type: WHO

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Cuộc kháng chiến chống thực dân Pháp xâm lược (EVENT) - 0.797
   2. Thực dân Pháp (ORGANIZATION) - 0.708
   3. Tham mưu trưởng quân viễn chinh Pháp (PERSON) - 0.699
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 19 total entities

 HYBRID SCORING & RANKING...
   Top 10 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 14 relationships

 Retrieval complete!


In [6]:
# Display top entities
print("\n=== TOP RETRIEVED ENTITIES ===")
for i, entity in enumerate(context['top_entities'], 1):
    print(f"\n{i}. {entity['name']} ({entity['type']})")
    print(f"   Score: {entity['score']:.4f}")
    print(f"   - Vector: {entity['vector_score']:.3f}")
    print(f"   - Graph: {entity['graph_score']:.3f}")
    print(f"   - Type: {entity['type_score']:.3f}")
    if entity.get('description'):
        print(f"   Description: {entity['description'][:80]}...")


=== TOP RETRIEVED ENTITIES ===

1. Tham mưu trưởng quân viễn chinh Pháp (PERSON)
   Score: 0.8496
   - Vector: 0.699
   - Graph: 1.000
   - Type: 1.000

2. De Gaulle (PERSON)
   Score: 0.8442
   - Vector: 0.688
   - Graph: 1.000
   - Type: 1.000

3. Thực dân Pháp (ORGANIZATION)
   Score: 0.7041
   - Vector: 0.708
   - Graph: 1.000
   - Type: 0.700
   Description: Kẻ thù...

4. Bọn thực dân Pháp (ORGANIZATION)
   Score: 0.6914
   - Vector: 0.683
   - Graph: 1.000
   - Type: 0.700
   Description: Quân đội Pháp...

5. Cuộc kháng chiến chống thực dân Pháp xâm lược (EVENT)
   Score: 0.5484
   - Vector: 0.797
   - Graph: 1.000
   - Type: 0.300
   Description: Chiến tranh...

6. Sainteny (PERSON)
   Score: 0.4799
   - Vector: 0.000
   - Graph: 0.500
   - Type: 1.000
   Description: Đại diện của Pháp...

7. Chính phủ Hồ Chí Minh (ORGANIZATION)
   Score: 0.4062
   - Vector: 0.000
   - Graph: 0.500
   - Type: 0.700

8. Cách mạng Việt Nam (ORGANIZATION)
   Score: 0.4062
   - Vector: 0.000
   - G

In [7]:
# Display relationships
print("\n=== KEY RELATIONSHIPS ===")
for i, rel in enumerate(context['relationships'][:10], 1):
    arrow = "→" if rel['direction'] == 'outgoing' else "←"
    print(f"{i}. {rel['source']} {arrow} [{rel['type']}] {arrow} {rel['target']}")
    if rel.get('description'):
        print(f"   {rel['description'][:60]}...")


=== KEY RELATIONSHIPS ===
1. Tham mưu trưởng quân viễn chinh Pháp → [HOẠT_ĐỘNG_CÙNG] → Sainteny
   hoạt động cùng Sainteny...
2. Thực dân Pháp → [TIÊU_DIỆT] → Việt Minh
   loại trừ bộ đội chủ lực cùa Việt Minh ra khỏi vòng chiến đấu...
3. Thực dân Pháp → [BẮT] → Chính phủ Hồ Chí Minh
   bắt Chính phủ Hồ Chí Minh...
4. Thực dân Pháp → [PHÁ_TAN] → Việt Minh
   phá tan cơ quan đầu não kháng chiến...
5. Thực dân Pháp → [TRIẾT_TIÊU] → Việt Minh
   triệt tiêu mọi tiềm lực kháng chiến của Việt Minh...
6. Thực dân Pháp → [NGĂN_CHẶN_MỐI_QUAN_HỆ] → Cách mạng Việt Nam
   ngăn chặn mọi mối quan hệ giữa Cách mạng Việt Nam và Cách mạ...
7. Bọn thực dân Pháp → [KHÁNG_CHIẾN] → Quốc gia tự vệ
   Kháng chiến Quốc gia tự vệ...
8. Sainteny ← [HOẠT_ĐỘNG_CÙNG] ← Tham mưu trưởng quân viễn chinh Pháp
   hoạt động cùng Sainteny...
9. Chính phủ Hồ Chí Minh ← [BẮT] ← Thực dân Pháp
   bắt Chính phủ Hồ Chí Minh...
10. Cách mạng Việt Nam ← [NGĂN_CHẶN_MỐI_QUAN_HỆ] ← Thực dân Pháp
   ngăn chặn mọi mối quan hệ giữa C

## Compare with Individual Approaches

So sánh Hybrid vs Vector-only vs Graph-only

In [8]:
test_question = "Chiến dịch quân sự quan trọng nhất"

print(f"Question: {test_question}\n")
print("="*70)

# 1. Vector Search only
print("\n VECTOR SEARCH ONLY (Bước 2):")
vector_results = embeddings.semantic_search(test_question, top_k=5)
for i, r in enumerate(vector_results, 1):
    print(f"   {i}. {r['name']} ({r['type']}) - {r['similarity']:.4f}")

# 2. Graph Search only (starting from most relevant)
print("\n GRAPH TRAVERSAL ONLY (Bước 1):")
if vector_results:
    seed_entity = vector_results[0]['name']
    print(f"   Starting from: {seed_entity}")
    graph_results = graph_query.get_related_entities(seed_entity, max_depth=1, limit=5)
    for i, r in enumerate(graph_results, 1):
        print(f"   {i}. {r['name']} ({r['type']}) - distance: {r['distance']}")

# 3. Hybrid Retrieval
print("\n  HYBRID RETRIEVAL (Bước 3):")
hybrid_context = hybrid.retrieve(test_question, top_k=5, expansion_depth=1)
for i, e in enumerate(hybrid_context['top_entities'], 1):
    print(f"   {i}. {e['name']} ({e['type']}) - {e['score']:.4f}")

print("\nHybrid combines best of both: semantic relevance + graph context!")

Question: Chiến dịch quân sự quan trọng nhất


 VECTOR SEARCH ONLY (Bước 2):
   1. phong tỏa (EVENT) - 0.8281
   2. Chiến dịch Biên giới (EVENT) - 0.8178
   3. chiến tranh du kích (EVENT) - 0.8109
   4. tấn công quấy rối (EVENT) - 0.7948
   5. Thập vạn đại sơn (EVENT) - 0.7935

 GRAPH TRAVERSAL ONLY (Bước 1):
   Starting from: phong tỏa

  HYBRID RETRIEVAL (Bước 3):
Query: Chiến dịch quân sự quan trọng nhất
Question type: WHAT

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. phong tỏa (EVENT) - 0.828
   2. Chiến dịch Biên giới (EVENT) - 0.818
   3. chiến tranh du kích (EVENT) - 0.811
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 1 total entities

 HYBRID SCORING & RANKING...
   Top 5 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 0 relationships

 Retrieval complete!
   1. phong tỏa (EVENT) - 0.9141
   2. Chiến dịch Biên giới (EVENT) - 0.9089
   3. chiến tranh du kích (EVENT) - 0.9055
   4. tấn công quấy rối (EVENT) - 0.8974
   5. Thập vạn đại sơn 

## Test Different Question Types

In [9]:
# Test với các loại câu hỏi khác nhau
test_questions = [
    "Ai là lãnh đạo quân đội?",                    # WHO
    "Chiến dịch diễn ra ở đâu?",                   # WHERE
    "Kháng chiến bắt đầu khi nào?",                # WHEN
    "Sự kiện quan trọng nhất là gì?",              # WHAT
]

for question in test_questions:
    print(f"\n{'='*70}")
    print(f" {question}")
    print(f"{'='*70}")
    
    # Get question type
    q_type = hybrid.infer_question_type(question)
    print(f"Question type detected: {q_type}")
    
    # Retrieve
    results = hybrid.retrieve_simple(question, top_k=5)
    
    print(f"\nTop 5 results:")
    for i, entity in enumerate(results, 1):
        print(f"  {i}. {entity['name']} ({entity['type']}) - {entity['score']:.4f}")


 Ai là lãnh đạo quân đội?
Question type detected: WHO
Query: Ai là lãnh đạo quân đội?
Question type: WHO

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Tường Giới Thạch (PERSON) - 0.853
   2. Bộ Chi huy (ORGANIZATION) - 0.733
   3. Tướng Lư Hán (PERSON) - 0.732
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 10 total entities

 HYBRID SCORING & RANKING...
   Top 5 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 2 relationships

 Retrieval complete!

Top 5 results:
  1. Tường Giới Thạch (PERSON) - 0.7835
  2. Tướng Lư Hán (PERSON) - 0.7593
  3. Bộ Chi huy (ORGANIZATION) - 0.7165
  4. Bộ Tổng chi huy (ORGANIZATION) - 0.7138
  5. Tổng Tư lệnh (ORGANIZATION) - 0.7134

 Chiến dịch diễn ra ở đâu?
Question type detected: WHERE
Query: Chiến dịch diễn ra ở đâu?
Question type: WHERE

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Chiến dịch Biên giới (EVENT) - 0.608
   2. Công trái kháng chiến (EVENT) - 0.586
   3. cuộc kháng chiến (EVENT) - 0.578

## Test with Graph Paths

Include paths between entities để hiểu connections

In [10]:
# Test với paths enabled
question = "Mối liên hệ giữa các lãnh đạo và chiến dịch"

context_with_paths = hybrid.retrieve(
    question=question,
    top_k=8,
    vector_top_k=5,
    expansion_depth=2,
    include_paths=True
)

print("\n=== PATHS BETWEEN KEY ENTITIES ===")
for i, path in enumerate(context_with_paths['paths'], 1):
    print(f"\nPath {i}: {path['source']} → {path['target']}")
    print(f"  Length: {path['path_length']}")
    print(f"  Route: ", end="")
    for j, node in enumerate(path['nodes']):
        print(node['name'], end="")
        if j < len(path['relationships']):
            print(f" --[{path['relationships'][j]['type']}]--> ", end="")
    print()

Query: Mối liên hệ giữa các lãnh đạo và chiến dịch
Question type: WHO

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Đàng (PERSON) - 0.688
   2. Bộ chi huy chiến dịch (ORGANIZATION) - 0.681
   3. Đảng ủy Mặt trận (ORGANIZATION) - 0.675
   ... and 2 more

 GRAPH EXPANSION (depth=2)...
   Expanded to 8 total entities

 HYBRID SCORING & RANKING...
   Top 8 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 10 relationships

 FINDING PATHS...
   Found 1 paths

 Retrieval complete!

=== PATHS BETWEEN KEY ENTITIES ===

Path 1: Bộ chi huy chiến dịch → Đảng ủy Mặt trận
  Length: 1
  Route: Bộ chi huy chiến dịch --[NHẬN_ĐỊNH]--> Đảng ủy Mặt trận


## Test Different Expansion Depths

In [11]:
# Compare different expansion depths
question = "Các tổ chức liên quan đến kháng chiến"

for depth in [1, 2]:
    print(f"\n{'='*70}")
    print(f"Expansion depth: {depth}")
    print(f"{'='*70}")
    
    context = hybrid.retrieve(
        question=question,
        top_k=10,
        expansion_depth=depth
    )
    
    stats = context['retrieval_stats']
    print(f"Seed entities: {stats['seed_entities']}")
    print(f"Expanded entities: {stats['expanded_entities']}")
    print(f"Final entities: {stats['final_entities']}")
    print(f"Relationships: {stats['relationships']}")
    
    print(f"\nTop 5 entities:")
    for i, e in enumerate(context['top_entities'][:5], 1):
        print(f"  {i}. {e['name']} ({e['type']}) - {e['score']:.4f}")


Expansion depth: 1
Query: Các tổ chức liên quan đến kháng chiến
Question type: DEFAULT

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Quỹ tham gia kháng chiến (ORGANIZATION) - 0.825
   2. Ủy ban kháng chiến chống cộng (ORGANIZATION) - 0.812
   3. Ứy ban kháng chiến hành chính (ORGANIZATION) - 0.808
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 7 total entities

 HYBRID SCORING & RANKING...
   Top 10 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 26 relationships

 Retrieval complete!
Seed entities: 5
Expanded entities: 7
Final entities: 10
Relationships: 26

Top 5 entities:
  1. Quỹ tham gia kháng chiến (ORGANIZATION) - 0.6626
  2. Ủy ban kháng chiến chống cộng (ORGANIZATION) - 0.6559
  3. Ứy ban kháng chiến hành chính (ORGANIZATION) - 0.6539
  4. Ủy ban kháng chiến huyện (ORGANIZATION) - 0.6460
  5. Chính phủ kháng chiến (ORGANIZATION) - 0.6427

Expansion depth: 2
Query: Các tổ chức liên quan đến kháng chiến
Question type: DEFAULT

 VECTOR SEAR

## Format Context for LLM

Xem context được format như thế nào để feed vào LLM

In [12]:
# Retrieve và format context
question = "Tóm tắt về Chiến dịch Điện Biên Phủ"

context = hybrid.retrieve(question, top_k=10)

# Format for LLM
formatted_context = hybrid.format_context_for_llm(context, max_entities=8)

print("\n" + "="*70)
print("FORMATTED CONTEXT FOR LLM:")
print("="*70)
print(formatted_context)

# This context would be used in Bước 6 (LLM Integration)

Query: Tóm tắt về Chiến dịch Điện Biên Phủ
Question type: WHAT

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. Quán triệt (EVENT) - 0.672
   2. phá tề (EVENT) - 0.628
   3. Chiến dịch Biên giới (EVENT) - 0.607
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 3 total entities

 HYBRID SCORING & RANKING...
   Top 8 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 9 relationships

 Retrieval complete!

FORMATTED CONTEXT FOR LLM:
=== RETRIEVED CONTEXT ===

ENTITIES:
1. Quán triệt (EVENT)
   Thực hiện chủ trương
   [Relevance: 0.836]

2. phá tề (EVENT)
   Hoạt động phá tề
   [Relevance: 0.814]

3. Chiến dịch Biên giới (EVENT)
   Chiến dịch quân sự
   [Relevance: 0.803]

4. VIỆN SỬ HỌC (ORGANIZATION)
   [Relevance: 0.601]

5. Tứ gia liên báo (ORGANIZATION)
   [Relevance: 0.600]

6. Quân và dân ta (ORGANIZATION)
   Người dân và lực lượng vũ trang Việt Nam
   [Relevance: 0.366]

7. du kích (GROUP)
   Nhóm người hoạt động quân sự phi tập trung
   [Relevance: 0.

##  Analyze Retrieval Statistics

In [13]:
# Get detailed statistics
stats = hybrid.get_statistics(context)

print("=== RETRIEVAL STATISTICS ===")
print(f"\nTotal entities: {stats['total_entities']}")
print(f"Average score: {stats['avg_score']:.4f}")
print(f"Relationships: {stats['relationships']}")
print(f"Paths: {stats['paths']}")
print(f"Question type: {stats['question_type']}")

print(f"\nEntity type distribution:")
for etype, count in stats['entity_types'].items():
    print(f"  {etype}: {count}")

=== RETRIEVAL STATISTICS ===

Total entities: 8
Average score: 0.5761
Relationships: 9
Paths: 0
Question type: WHAT

Entity type distribution:
  EVENT: 3
  ORGANIZATION: 3
  GROUP: 1
  LOCATION: 1


## Test Complex Multi-Part Questions

In [14]:
# Test với câu hỏi phức tạp
complex_questions = [
    "Ai chỉ huy chiến dịch Điện Biên Phủ và kết quả như thế nào?",
    "Mối quan hệ giữa Hồ Chí Minh và các tổ chức cách mạng",
    "Các sự kiện quan trọng trong giai đoạn 1945-1954",
]

for question in complex_questions:
    print(f"\n{'='*70}")
    print(f"{question}")
    print(f"{'='*70}")
    
    context = hybrid.retrieve(
        question=question,
        top_k=12,
        vector_top_k=6,
        expansion_depth=2
    )
    
    print(f"\nTop 5 entities:")
    for i, e in enumerate(context['top_entities'][:5], 1):
        print(f"  {i}. {e['name']} ({e['type']}) - {e['score']:.4f}")
    
    print(f"\nKey relationships: {len(context['relationships'])}")
    for i, rel in enumerate(context['relationships'][:3], 1):
        print(f"  {i}. {rel['source']} --[{rel['type']}]--> {rel['target']}")


Ai chỉ huy chiến dịch Điện Biên Phủ và kết quả như thế nào?
Query: Ai chỉ huy chiến dịch Điện Biên Phủ và kết quả như thế nào?
Question type: WHO

 VECTOR SEARCH (top 6)...
   Found 6 seed entities:
   1. Bộ chi huy chiến dịch (ORGANIZATION) - 0.617
   2. Lãnh đạo phong trào du kích Quảng Tây (PERSON) - 0.601
   3. Tập tài liệu về tình hình tổ chức bộ máy và hoạt động của chính phủ bù nhìn năm 1948-1952 (EVENT) - 0.598
   ... and 3 more

 GRAPH EXPANSION (depth=2)...
   Expanded to 9 total entities

 HYBRID SCORING & RANKING...
   Top 12 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 12 relationships

 Retrieval complete!

Top 5 entities:
  1. Lãnh đạo phong trào du kích Quảng Tây (PERSON) - 0.8006
  2. quân và dân ta (ORGANIZATION) - 0.6480
  3. Bộ chi huy chiến dịch (ORGANIZATION) - 0.5667
  4. Tập tài liệu về tình hình tổ chức bộ máy và hoạt động của chính phủ bù nhìn năm 1948-1952 (EVENT) - 0.4492
  5. Thập vạn đại sơn (EVENT) - 0.4371

Key relationships: 12
  1. Bộ chi 

## Benchmark Performance

In [15]:
import time

# Test performance với different configurations
test_question = "Lãnh đạo quân sự trong kháng chiến"

configs = [
    {"top_k": 5, "vector_top_k": 3, "depth": 1},
    {"top_k": 10, "vector_top_k": 5, "depth": 1},
    {"top_k": 15, "vector_top_k": 5, "depth": 2},
]

print("=== PERFORMANCE BENCHMARK ===")
print(f"Question: {test_question}\n")

for config in configs:
    start = time.time()
    
    context = hybrid.retrieve(
        question=test_question,
        top_k=config['top_k'],
        vector_top_k=config['vector_top_k'],
        expansion_depth=config['depth']
    )
    
    elapsed = time.time() - start
    
    stats = context['retrieval_stats']
    
    print(f"\nConfig: {config}")
    print(f"  Time: {elapsed:.2f}s")
    print(f"  Entities: {stats['final_entities']}")
    print(f"  Relationships: {stats['relationships']}")

=== PERFORMANCE BENCHMARK ===
Question: Lãnh đạo quân sự trong kháng chiến

Query: Lãnh đạo quân sự trong kháng chiến
Question type: WHO

 VECTOR SEARCH (top 3)...
   Found 3 seed entities:
   1. quân và dân ta (ORGANIZATION) - 0.824
   2. Tường Giới Thạch (PERSON) - 0.807
   3. Quân khu Thủ đô (ORGANIZATION) - 0.786

 GRAPH EXPANSION (depth=1)...
   Expanded to 6 total entities

 HYBRID SCORING & RANKING...
   Top 5 entities selected

 EXTRACTING RELATIONSHIPS...
   Found 4 relationships

 Retrieval complete!

Config: {'top_k': 5, 'vector_top_k': 3, 'depth': 1}
  Time: 6.83s
  Entities: 5
  Relationships: 4
Query: Lãnh đạo quân sự trong kháng chiến
Question type: WHO

 VECTOR SEARCH (top 5)...
   Found 5 seed entities:
   1. quân và dân ta (ORGANIZATION) - 0.824
   2. Tường Giới Thạch (PERSON) - 0.807
   3. Quân khu Thủ đô (ORGANIZATION) - 0.786
   ... and 2 more

 GRAPH EXPANSION (depth=1)...
   Expanded to 9 total entities

 HYBRID SCORING & RANKING...
   Top 10 entities selected

 